## Recreating a notebook to learn: Daily AQI Geospatial Analysis

Link to the original notebook: https://www.kaggle.com/code/azminetoushikwasi/daily-aqi-geospatial-analysis

This is about Air Quality Index (AQI), which describes the degree of pollution in the air

The original notebook has a nice graphic but I'll just practice using Markdown to import a picture:

![Cat in a strawberry hat](img/Wicked_Background_Final.png)

AQIs:

- 0 - 50 : Good
- 51 - 100: Satisfactory
- 101 - 200: Moderate
- 201 - 300: Poor
- 301 - 400: Very Poor
- 401 - 500: Severe

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd 
import os # provides a portable way of using operating system dependent functionality
import seaborn as sns # Python data visualization library based on matplotlib
import matplotlib.pyplot as plt
from scipy import stats # scientific computation library that uses NumPy underneath
import warnings # built-in module designed to manage and control warning messages generated during program execution
import json

from bs4 import BeautifulSoup as BeautifulSoup # Python library designed for parsing HTML and XML documents, making it a popular tool for web scraping
import pandas as pd # why import pandas twice? perhaps mistake?
from urllib.request import Request, urlopen # urllib is a package that collects several modules for working with URLs: urllib.request for opening and reading URLs
import re # provides support for regular expressions, which are powerful tools for pattern matching and manipulation of strings
import time
import random
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

"""
Bokeh is a Python interactive visualization library that enables the creation of elegant and interactive plots, 
dashboards, and data applications for modern web browsers
"""
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer,all_palettes
import geopandas as gpd
import plotly.express as px

In [7]:
## Creating some lists

aqi_name_list=['Albania', 'Algeria', 'Andorra', 'Angola', 'Argentina', 'Armenia', 'Australia',
               'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Bermuda',
               'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cambodia',
               'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China',
               'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic',
               'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'French Guiana',
               'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Grenada', 'Guadeloupe',
               'Guam', 'Guatemala', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran',
               'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan',
               'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Liberia', 'Liechtenstein',
               'Lithuania', 'Luxembourg', 'Macao', 'Macedonia', 'Madagascar', 'Malaysia', 'Malta', 'Martinique',
               'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Myanmar', 'Nepal', 'Netherlands', 'New Caledonia',
               'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Palestinian Territory', 'Peru', 'Philippines', 'Poland',
               'Portugal', 'Puerto Rico', 'Qatar', 'Reunion', 'Romania', 'Russia', 'San Marino', 'Saudi Arabia', 'Senegal',
               'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Sudan',
               'Sweden', 'Switzerland', 'Taiwan', 'Tajikistan', 'Thailand', 'Togo', 'Trinidad and Tobago', 'Turkey', 'Turkmenistan',
               'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom of Great Britain and Northern Ireland', 'United States of America', 
               'Uzbekistan', 'Vatican', 'Venezuela',
               'Vietnam', 'Zambia']
aqi_list =['/country/albania/', '/country/algeria/', '/country/andorra/', 
 '/country/angola/', '/country/argentina/', '/country/armenia/', '/country/australia/', 
 '/country/austria/', '/country/azerbaijan/', '/country/bahrain/', '/country/bangladesh/', 
 '/country/belarus/', '/country/belgium/', '/country/belize/', '/country/bermuda/', '/country/bolivia/', 
 '/country/bosnia-and-herzegovina/', '/country/brazil/', '/country/brunei/', '/country/bulgaria/', '/country/burkina-faso/',
 '/country/cambodia/', '/country/canada/', '/country/cape-verde/', '/country/cayman-islands/', '/country/central-african-republic/', 
 '/country/chad/', '/country/chile/', '/country/china/', '/country/colombia/', '/country/costa-rica/', '/country/croatia/', 
 '/country/cyprus/', '/country/czech-republic/', '/country/denmark/', '/country/dominican-republic/', '/country/ecuador/', 
 '/country/egypt/', '/country/el-salvador/', '/country/estonia/', '/country/ethiopia/', '/country/finland/', '/country/france/', 
 '/country/french-guiana/', '/country/gabon/', '/country/gambia/', '/country/georgia/', '/country/germany/', '/country/ghana/', 
 '/country/gibraltar/', '/country/greece/', '/country/grenada/', '/country/guadeloupe/', '/country/guam/', '/country/guatemala/', 
 '/country/honduras/', '/country/hong-kong/', '/country/hungary/', '/country/iceland/', '/country/india/', '/country/indonesia/',
 '/country/iran/', '/country/iraq/', '/country/ireland/', '/country/israel/', '/country/italy/', '/country/ivory-coast/', 
 '/country/japan/', '/country/jersey/', '/country/jordan/', '/country/kazakhstan/', '/country/kenya/', '/country/kosovo/',
 '/country/kuwait/', '/country/kyrgyzstan/', '/country/laos/', '/country/latvia/', '/country/lebanon/', '/country/liberia/',
 '/country/liechtenstein/', '/country/lithuania/', '/country/luxembourg/', '/country/macao/', '/country/macedonia/', 
 '/country/madagascar/', '/country/malaysia/', '/country/malta/', '/country/martinique/', '/country/mexico/', '/country/moldova/',
 '/country/monaco/', '/country/mongolia/', '/country/montenegro/', '/country/myanmar/', '/country/nepal/', '/country/netherlands/',
 '/country/new-caledonia/', '/country/new-zealand/', '/country/nigeria/', '/country/norway/', '/country/pakistan/', 
 '/country/palestinian-territory/', '/country/peru/', '/country/philippines/', '/country/poland/', '/country/portugal/', 
 '/country/puerto-rico/', '/country/qatar/', '/country/reunion/', '/country/romania/', '/country/russia/', '/country/san-marino/',
 '/country/saudi-arabia/', '/country/senegal/', '/country/serbia/', '/country/singapore/', '/country/slovakia/', '/country/slovenia/',
 '/country/south-africa/', '/country/south-korea/', '/country/spain/', '/country/sri-lanka/', '/country/sudan/', '/country/sweden/', 
 '/country/switzerland/', '/country/taiwan/', '/country/tajikistan/', '/country/thailand/', '/country/togo/', '/country/trinidad-and-tobago/', 
 '/country/turkey/', '/country/turkmenistan/', '/country/uganda/', '/country/ukraine/', '/country/united-arab-emirates/', '/country/united-kingdom/', 
 '/country/united-states/', '/country/uzbekistan/', '/country/vatican/', '/country/venezuela/', '/country/vietnam/', '/country/zambia/']

In [13]:
## trying out request and urlopen with just one webpage - for Uzbekistan!
req = Request('https://aqicn.org/country/uzbekistan', headers = {'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
webpage

b'\xef\xbb\xbf<!doctype html><html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en" itemscope itemtype="http://schema.org/LocalBusiness" ng-app="waqi"> <head> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /> <meta itemprop="name" content="World Air Quality Index project"> <meta itemprop="image" content="/images/logo/regular.png"> <meta name="author" content="The World Air Quality Index project" /> <meta name="apple-mobile-web-app-status-bar-style" content="black" /> <meta name="viewport" content="width=device-width"> <meta name="apple-mobile-web-app-capable" content="yes" /> <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent" /> <title>Air Pollution in Uzbekistan</title> <link rel="shortcut icon" href="/images/favicon.ico" /> <link rel="apple-touch-icon"  sizes="114x114"  href="/images/iOS/ico_114.png"/> <link rel="apple-touch-icon"  sizes="72x72"  href="/images/iOS/ico_72.png"/> <link rel="apple-touch-icon"  href="/images/iOS

In [11]:
soup = BeautifulSoup(webpage, 'html.parser')
soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/LocalBusiness" lang="en" ng-app="waqi" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"> <head> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <meta content="World Air Quality Index project" itemprop="name"/> <meta content="/images/logo/regular.png" itemprop="image"/> <meta content="The World Air Quality Index project" name="author"> <meta content="black" name="apple-mobile-web-app-status-bar-style"> <meta content="width=device-width" name="viewport"/> <meta content="yes" name="apple-mobile-web-app-capable"> <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/> <title>Air Pollution in Uzbekistan</title> <link href="/images/favicon.ico" rel="shortcut icon"/> <link href="/images/iOS/ico_114.png" rel="apple-touch-icon" sizes="114x114"/> <link href="/images/iOS/ico_72.png" rel="apple-touch-icon" sizes="72x72"/> <link href="/images/iOS/ico_57.png" rel="apple-touch-icon"/> <link 

In [21]:
div_aqi_value_lst = soup.select('div.country-aqi-value')
div_aqi_value_lst

[<div class="country-aqi-value" style="background-color: #8abe4a;color:#000000">27</div>]

In [22]:
## Getting the AQI just for Uzbekistan still before putting all the countries through the for loop

aqi_value_uzbekistan = div_aqi_value_lst[0].text

print(div_aqi_value_lst[0])
# print(products[1]) # this goes out of range - only element in the products list is the div
print(div_aqi_value_lst[0].text)
print(aqi_value_uzbekistan)

<div class="country-aqi-value" style="background-color: #8abe4a;color:#000000">27</div>
27
27


In [23]:
## Now getting AQI impact just for Uzbekistan

div_aqi_impact_lst = soup.select('div.country-aqi-impact')
aqi_status_uzbekistan = div_aqi_impact_lst[0].text

print(div_aqi_impact_lst)
print(aqi_status_uzbekistan)

[<div class="country-aqi-impact" style="color: #009966;text-shadow: 1px 1px 0 #000000">Good</div>]
Good


In [24]:
aqi_values = []
aqi_statuses = []

for i in range(len(aqi_list)):
    country_url_part = aqi_list[i]
    country_name = aqi_name_list[i]

    url = 'https://aqicn.org' + country_url_part
    req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    
    soup = BeautifulSoup(webpage, 'html.parser')

    div_aqi_value_lst = soup.select('div.country-aqi-value')
    aqi_value = div_aqi_value_lst[0].text
    aqi_values.append(aqi_value)

    div_aqi_impact_lst = soup.select('div.country-aqi-impact')
    aqi_status = div_aqi_impact_lst[0].text
    aqi_statuses.append(aqi_status)

    print(str((i + 1) * 100 / len(aqi_list)) + "% done") # guessing this returns percent done over time

    time.sleep(random.randint(0, 2)) # time.sleep() can let the server take breaks during computing and avoid overwhelming the server
 

0.704225352112676% done
1.408450704225352% done
2.112676056338028% done
2.816901408450704% done
3.5211267605633805% done
4.225352112676056% done
4.929577464788732% done
5.633802816901408% done
6.338028169014085% done
7.042253521126761% done
7.746478873239437% done
8.450704225352112% done
9.154929577464788% done
9.859154929577464% done
10.56338028169014% done
11.267605633802816% done
11.971830985915492% done
12.67605633802817% done
13.380281690140846% done
14.084507042253522% done
14.788732394366198% done
15.492957746478874% done
16.197183098591548% done
16.901408450704224% done
17.6056338028169% done
18.309859154929576% done
19.014084507042252% done
19.718309859154928% done
20.422535211267604% done
21.12676056338028% done
21.830985915492956% done
22.535211267605632% done
23.239436619718308% done
23.943661971830984% done
24.64788732394366% done
25.35211267605634% done
26.056338028169016% done
26.760563380281692% done
27.464788732394368% done
28.169014084507044% done
28.87323943661972% d

In [29]:
##### NEED TO GO THROUGH AND UNDERSTAND THE BELOW
today_date = datetime.today().strftime('%Y-%m-%d')
today_data = {
    "Country":aqi_name_list,
    "Status":aqi_statuses,
    "AQI Value":aqi_values
    }

today = pd.DataFrame(today_data)
today.Status = today.Status.str.replace('Unhealthyfor Sensitive Groups','Unhealthy for Sensitive Groups')
today.Status = today.Status.str.replace('VeryUnhealthy','Very Unhealthy')
today["AQI Value"] = today["AQI Value"].str.replace('-','0')

today = today.drop(today[today.Status == 'no data'].index)
today = today.reset_index()

today_list = [today_date for i in range(len(today ))]

data_date_today = pd.DataFrame({"Date":today_list,
                              "Country":today["Country"],
                              "Status":today.Status,
                              "AQI Value":today["AQI Value"]
                        })
# Date
try:
    data_date=pd.read_csv('/kaggle/input/aqi-air-quality-index-scheduled-daily-update/data_date.csv')
    data_date=data_date.append(data_date_today)
    data_date.to_csv('data_date.csv',index=False)
except:
    data_date=data_date_today.copy()
    data_date.to_csv('data_date.csv',index=False)

In [28]:
gdf = gpd.read_file(
    "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
)
